This notebook is used to plot the metrics of the reads that are produced in the simulation study. The goal is to simulate reads with different characteristics (depth, readlength, accuracy) and produce assemblies. The read datasets will be characterized by:
- Number of reads 
- Readlength distribution
- Accuracy distribution
-

In [2]:
from matplotlib import pyplot as plt 
import numpy as np 
import bionumpy as bnp
import pandas as pd
from scipy import stats
import seaborn as sns
import os
from os import environ

In [ ]:
# We want to plot:
# - per-read accuracy
# - readlength distribution
# PBSIM3 does not output the accuracy in the fastq file so we will have to use the alignment of reads to the reference
# In the snakemake pipeline minimap2 was used to align the reads back to their reference, resulting in a bam file.
# Here htsbox samview is used to translate that back to a paf file 
# For paf file format see: https://cran.r-project.org/web/packages/pafr/vignettes/Introduction_to_pafr.html
# This approach of using the alignment to calculate accuracy is also used by Ono-2022 (PBSIM3 paper): "Accuracy [...] was computed from alignments between the reads and their reference genome"

In [5]:
# Function to extract the fields we need from the .paf file
# See also https://cran.r-project.org/web/packages/pafr/vignettes/Introduction_to_pafr.html
# This is working for the paf files created by htsbox samview! (For paf files created by minimap2 directly, length is len(fields[9]) and nm is int(fields[11].split(':')[2])
def parse_paf(file_path):
    alignments = []
    with open(file_path, 'r') as file:
        for line in file:
            if line[0] == 'S':    
                fields = line.strip().split('\t')
                #Check if this is not an entry that gives "*" as sequence
                if len(fields[9])>1:
                    alignment = {
                        'query_name': fields[0],
                        'query_length': int(fields[1]),
                        'num_matches': int(fields[9]),
                        'alen': int(fields[10])
                        #'num_mismatches': int(fields[12].split(':')[2])
                    }
                alignments.append(alignment)
    #for some reason there are duplicates (due to secondary alignments?) so we need to remove these
    unique_alignments = [dict(t) for t in {tuple(d.items()) for d in alignments}]
    return unique_alignments

In [8]:
### ONT make paf file ###
!find "/home/uitte01p/experimental/test_for_sim_study/simulation_output/ONT" -name *.fq.gz > ont_reads_filepaths.txt
with open("ont_reads_filepaths.txt") as filepaths:
    for line in filepaths:
        line = line.rstrip()
        bam_line = line.replace(".fq.gz",".bam")
        paf_line = line.replace(".fq.gz",".paf")
        environ['bam_line'] = bam_line
        environ['paf_line'] = paf_line
        !htsbox samview -p "$bam_line" > "$paf_line"

In [9]:
!find "/home/uitte01p/experimental/test_for_sim_study/simulation_output/ONT" -name *.paf > ont_paf_files.txt

In [10]:
### ONT plots ###
# Read in all the paf files and plot their readlength and accuracy distributions
# For the definition of BLAST identity (PAF file column 10 divided by 11), see https://epi2me.nanoporetech.com/quality-scores/ and https://lh3.github.io/2018/11/25/on-the-definition-of-sequence-identity
!find "/home/uitte01p/experimental/test_for_sim_study/simulation_output/ONT" -name *.paf > ont_paf_files.txt
with open("ont_paf_files.txt") as pafs:
    for paf in pafs:
        paf = paf.rstrip()
        alignments = parse_paf(paf)
        all_accuracies=[]
        all_lengths = []
        ac_fig_filepath = paf.replace("_0001.paf",".blast_identity.png")
        rl_fig_filepath = paf.replace("_0001.paf",".readlengths.png")
        qc_line = paf.replace("_0001.paf",".accuracy_summary.txt")
        for alignment in alignments:
            accuracy = alignment['num_matches']/alignment['alen']
            all_accuracies.append(accuracy)
            all_lengths.append(alignment['query_length'])
        if "rl10000" in paf:
            colours = 'green'
        elif "rl25000" in paf:
            colours = 'red'
        elif "rl50000" in paf:
            colours = 'blue'
        ##Plot accuracies
        plt.figure()
        sns.histplot(all_accuracies,color=colours,edgecolor='white');
        plt.title("Per-read accuracy distribution")
        plt.xlabel("Accuracy")
        plt.ylabel("Count")
        plt.savefig(ac_fig_filepath)
        plt.close()
        ##Plot readlengths
        plt.figure()
        sns.histplot(all_lengths,bins=50,color=colours,edgecolor='white');
        plt.title("Readlength distribution")
        plt.xlabel("Readlength")
        plt.ylabel("Count")
        plt.xlim(0,100000)
        plt.savefig(rl_fig_filepath)
        plt.close()
        median_accuracy = np.median(all_accuracies)
        mean_accuracy = np.mean(all_accuracies)
        sd_accuracy = np.std(all_accuracies)
        with open(qc_line,'w') as qc_file:
            print('Median accuracy',median_accuracy,'\n','Mean accuracy',mean_accuracy,'\n','sd',sd_accuracy,file=qc_file)

Next up are the HiFi simulations (RSII produced by PBSIM3 then followed by CCS), plot readlength and accuracy distribution for each dataset in their own path

In [3]:
### HIFI make paf file ###
!find "/home/uitte01p/experimental/test_for_sim_study/simulation_output/HiFi" -name *.fastq.gz > hifi_reads_filepaths.txt
with open("hifi_reads_filepaths.txt") as filepaths:
    for line in filepaths:
        line = line.rstrip()
        bam_line = line.replace(".fastq.gz",".bam")
        paf_line = line.replace(".fastq.gz",".paf")
        environ['bam_line'] = bam_line
        environ['paf_line'] = paf_line
        !htsbox samview -p "$bam_line" > "$paf_line"

In [6]:
### HIFI plots ###
# Read in all the paf files and plot their readlength and accuracy distributions
# For the definition of BLAST identity (PAF file column 10 divided by 11), see https://epi2me.nanoporetech.com/quality-scores/ and https://lh3.github.io/2018/11/25/on-the-definition-of-sequence-identity
!find "/home/uitte01p/experimental/test_for_sim_study/simulation_output/HiFi/ccs" -name *.paf > hifi_paf_files.txt
with open("hifi_paf_files.txt") as pafs:
    for paf in pafs:
        paf = paf.rstrip()
        alignments = parse_paf(paf)
        all_accuracies=[]
        all_lengths = []
        ac_fig_filepath = paf.replace(".paf",".blast_identity.png")
        rl_fig_filepath = paf.replace(".paf",".readlenghts.png")
        qc_line = paf.replace(".paf",".accuracy_summary.txt")
        for alignment in alignments:
            accuracy = alignment['num_matches']/alignment['alen']
            all_accuracies.append(accuracy)
            all_lengths.append(alignment['query_length'])
        if "rl10000" in paf:
            colours = 'green'
        elif "rl17500" in paf:
            colours = 'red'
        elif "rl25000" in paf:
            colours = 'blue'
        ## Plot accuracy
        plt.figure()
        sns.histplot(all_accuracies,color=colours,edgecolor='white');
        plt.title("Per-read accuracy distribution")
        plt.xlabel("Accuracy")
        plt.ylabel("Count")
        plt.savefig(ac_fig_filepath)
        plt.close()
        ## Plot readlengths
        plt.figure()
        sns.histplot(all_lengths,bins=50,color=colours,edgecolor='white');
        plt.title("Readlength distribution")
        plt.xlabel("Readlength")
        plt.ylabel("Count")
        plt.xlim(0,100000)
        plt.savefig(rl_fig_filepath)
        plt.close()
        ## Write summary file of accuracies
        median_accuracy = np.median(all_accuracies)
        mean_accuracy = np.mean(all_accuracies)
        sd_accuracy = np.std(all_accuracies)
        with open(qc_line,'w') as qc_file:
            print('Median accuracy',median_accuracy,'\n','Mean accuracy',mean_accuracy,'\n','sd',sd_accuracy,file=qc_file)
!rm hifi_paf_files.txt

In [ ]:
# !find "/home/uitte01p/experimental/test_for_sim_study/simulation_output/HiFi" -name *.fastq > hifi_unzipped_reads_filepaths.txt
# with open("hifi_unzipped_reads_filepaths.txt") as filepaths:
#     for unzipped_line in filepaths:
#         unzipped_line = unzipped_line.rstrip()
#         zipped_line = unzipped_line.replace(".fastq",".fastq.gz")
#         environ['unzipped'] = unzipped_line
#         environ['zipped'] = zipped_line
#         !gzip -c "$unzipped" > "$zipped"

In [ ]:
### ONT READLENGTHS FROM FASTA (not used anymore because this can be done directly from the paf file, which we need for accuracy distribution anyways)
# Find all the paths to the simulated reads by PBSIM3 and generate a .readlengths.txt file that contains the counts of all readlengths
# !find "/home/uitte01p/experimental/test_for_sim_study/simulation_output/ONT" -name *.fq.gz > ont_reads_filepaths.txt
# with open("ont_reads_filepaths.txt") as filepaths:
#     for line in filepaths:
#         line = line.rstrip()
#         unzipped_line = line.replace(".gz","")
#         readlengths_filename = line.replace("_0001.fq.gz",".readlengths.txt")
#         environ['line'] = line
#         environ['readlengths_filename'] = readlengths_filename
#         environ['unzipped_line'] = unzipped_line
#         !gunzip -dkf "$line" 
# #See https://www.biostars.org/p/72433/
#         !awk 'NR % 4 ==2 {lengths[length($0)]++} END {for (len in lengths) print len, lengths[len]}' "$unzipped_line" > "$readlengths_filename"

In [ ]:
### ONT READLENGTHS
# For all the readlengths.txt files generated in the cell above, plot the readlength distribution and save in their own path.=
# See also https://bioinformatics.stackexchange.com/questions/45/read-length-distribution-from-fasta-file
# !find "/home/uitte01p/experimental/test_for_sim_study/simulation_output/ONT" -name *.readlengths.txt > readlength_distributions.txt
# with open("readlength_distributions.txt") as distributions:
#     for line in distributions:
#         line = line.rstrip()
#         rl_fig_filepath = line.replace(".txt",".png")
#         length_distribution = pd.read_csv(line, delimiter=' ', header=None)
#         if "rl10000" in line:
#             colours = 'green'
#         elif "rl25000" in line:
#             colours = 'red'
#         elif "rl50000" in line:
#             colours = 'blue'
#         plt.figure()
#         sns.histplot(length_distribution[0],bins=50,color=colours,edgecolor='white');
#         plt.title("Readlength distribution")
#         plt.xlabel("Readlength")
#         plt.ylabel("Count")
#         plt.xlim(0,100000)
#         plt.savefig(rl_fig_filepath)
#         plt.close()
# !rm readlength_distributions.txt

In [ ]:
# ### HIFI READLENGTHS
# # Find all the paths to the simulated reads by PBSIM3 and generate a .readlengths.txt file that contains the counts of all readlengths
# !find "/home/uitte01p/experimental/test_for_sim_study/simulation_output/HiFi" -name *.fastq.gz > hifi_reads_filepaths.txt
# with open("hifi_reads_filepaths.txt") as filepaths:
#     for line in filepaths:
#         line = line.rstrip()
#         unzipped_line = line.replace(".gz","")
#         readlengths_filename = line.replace(".fastq.gz",".readlengths.txt")
#         environ['line'] = line
#         environ['readlengths_filename'] = readlengths_filename
#         environ['unzipped_line'] = unzipped_line
#         !gunzip -dkf "$line" 
# #See https://www.biostars.org/p/72433/
#         !awk 'NR % 4 ==2 {lengths[length($0)]++} END {for (len in lengths) print len, lengths[len]}' "$unzipped_line" > "$readlengths_filename"

In [ ]:
# ### HIFI READLENGTHS
# # For all the readlengths.txt files generated in the cell above, plot the readlength distribution and save in their own path.=
# # See also https://bioinformatics.stackexchange.com/questions/45/read-length-distribution-from-fasta-file
# !find "/home/uitte01p/experimental/test_for_sim_study/simulation_output/HiFi" -name *.readlengths.txt > hifi_readlength_distributions.txt
# with open("hifi_readlength_distributions.txt") as distributions:
#     for line in distributions:
#         line = line.rstrip()
#         rl_fig_filepath = line.replace(".txt",".png")
#         length_distribution = pd.read_csv(line, delimiter=' ', header=None)
#         if "rl10000" in line:
#             colours = 'green'
#         elif "rl17500" in line:
#             colours = 'red'
#         elif "rl25000" in line:
#             colours = 'blue'
#         plt.figure()
#         sns.histplot(length_distribution[0],bins=50,color=colours,edgecolor='white');
#         plt.title("Readlength distribution")
#         plt.xlabel("Readlength")
#         plt.ylabel("Count")
#         plt.xlim(0,100000)
#         plt.savefig(rl_fig_filepath)
#         plt.close()
# !rm hifi_readlength_distributions.txt